In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import re
import os
import time
import urllib
import numpy as np
import pandas as pd
import requests
import multiprocessing
from pathlib import Path
from functools import partial
from PIL import Image
from tqdm import tqdm

In [2]:
df = pd.read_csv("D:\hackthons\ML-Challenge/artifacts\dataset/train.csv")

In [3]:
df.sample(10)

,image_link,group_id,entity_name,entity_value
72079,https://m.media-amazon.com/images/I/81-feS2rFO...,993359,item_weight,3 gram
179849,https://m.media-amazon.com/images/I/51QEfyJOuu...,675317,width,38.0 millimetre
148629,https://m.media-amazon.com/images/I/5158tQc7qe...,955522,width,30.9 millimetre
131436,https://m.media-amazon.com/images/I/415lSCcnnb...,658003,width,14.4 inch
143093,https://m.media-amazon.com/images/I/51CcwY27HG...,752266,width,1.5 inch
22691,https://m.media-amazon.com/images/I/718UYDNu97...,273748,item_weight,0.76 kilogram
77020,https://m.media-amazon.com/images/I/61U1WBhQb7...,558374,item_weight,108 gram
129394,https://m.media-amazon.com/images/I/71FQgXJL9r...,365637,item_weight,250.0 gram
188860,https://m.media-amazon.com/images/I/51SiZeHit9...,764618,depth,95.0 millimetre
32430,https://m.media-amazon.com/images/I/51BakaBOsm...,611510,maximum_weight_recommendation,300.0 pound


In [4]:
def create_placeholder_image(image_save_path):
    """
    Create a black placeholder image in case of a failed download.
    """
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        print(f"Error creating placeholder image: {e}")

In [5]:
def download_image(image_link, save_folder, retries=3, delay=3):
    """
    Download an image from a link and save it to a specified folder.
    """
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception as e:
            print(f"Error downloading image {image_link}: {e}")
            time.sleep(delay)
    
    create_placeholder_image(image_save_path)

In [6]:
def download_images(image_links, download_folder, allow_multiprocessing=True):
    """
    Download multiple images from a list of links and save them to a specified folder.
    """
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        # Limit the pool size to the number of available CPU cores
        pool_size = min(64, multiprocessing.cpu_count())
        with multiprocessing.Pool(processes=pool_size) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

In [17]:
download_folder = os.path.join('D:/hackthons/ML-Challenge/artifacts/dataset',"train")
if not os.path.exists(download_folder):
    os.makedirs(download_folder)
download_images(df['image_link'],download_folder)



  0%|          | 0/263859 [00:00<?, ?it/s]

In [9]:
df['image_link'][0]

'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'